# Setup

In [1]:
%matplotlib notebook
%load_ext autoreload
%autoreload 2

import ipywidgets as widgets
from ipywidgets import interact
from IPython.display import HTML,clear_output
from importlib import reload

import itertools
import numpy as np
from numba import njit,double
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import plotly.express as px
import plotly.graph_objects as go

from importData import import_all,generate_diffuse_tether_trajectories
import displayData
import trajAnalysis
import bayesianTools
from model_utils import GenerationMode
from utils import notebook_setup_widgets

# Import a dataframe

## Import experimental data

*Note*: The first import after will include reading from .csv and saving to a binary feather file and might take a few minutes.

#### Select experiments and/or particles with the widgets:

In [ ]:
full_df = import_all();df = full_df.copy();
expSelectWidget,rdmEnableWidget,rdmSameVideoWidget,keepStuckWidget,rdmSelectionWidget,specificEnableWidget,specificParticleSelectionWidget,buttonFilterWidget,filterDataframesFun=notebook_setup_widgets(widgets)
expSelectWidget.options = full_df.experiment.unique(); specificParticleSelectionWidget.min = full_df.particle.min(); specificParticleSelectionWidget.max = full_df.particle.max(); buttonFilterWidget.on_click(lambda b: filterDataframesFun(full_df,df))    
display(expSelectWidget,widgets.HBox((widgets.VBox((rdmEnableWidget,keepStuckWidget)),widgets.VBox((rdmSelectionWidget,rdmSameVideoWidget)))),widgets.HBox((specificEnableWidget,specificParticleSelectionWidget)),buttonFilterWidget)

### Alternatively, create a synthetic dataframe:

In [ ]:
N_steps = 1000
N_particle=4

dt=1./30
T_stick= 200*dt
T_unstick= 300*dt
D= 0.3333
A= 0.04444/10

generation_mode=GenerationMode.DONT_FORCE #keep this
init_S=None #random
do_post_processing=False #calculate some extra statistics, this takes a short while
undersample_ratio=0#0.1
save_files=False
is_parallel=False #relevant only if do_post_processing==True

df=generate_diffuse_tether_trajectories(T_stick, T_unstick, D, A,dt, N_steps,  N_particle, init_S,
                                     do_post_processing,undersample_ratio, save_files, generation_mode, is_parallel)

true_df = df.copy()

#### Check optimization stuff

In [ ]:
import time
X_arr_list = bayesianTools.extract_X_arr_list_from_df(df)
dt_list = bayesianTools.extract_dt_list_from_df(df)

def costFun(params):
#     TT_stick,TT_unstick,DD,AA = params
    DD,AA = params
    TT_stick = T_stick
    TT_unstick = T_unstick
    t = time.time()
    L = bayesianTools.multiple_trajectories_likelihood(X_arr_list, dt_list, TT_stick, TT_unstick, DD, AA,
                                     is_parallel=False)
    print("Params=[{},{},{},{}]; L={}; In {} sec".format(
    np.round(TT_stick,0),np.round(TT_unstick,0),np.round(DD,9),np.round(AA,9),np.round(L,9),np.round(time.time()-t,1)))
    return -L

In [ ]:
import scipy

res = scipy.optimize.minimize(costFun, [D*2,A/2])

#### Check that the Viterbi paths are close enough to the true paths

In [ ]:
N_rows = int(np.ceil(N_particle/2))
fig,ax=plt.subplots(N_rows,2)
reload(trajAnalysis)
reload(bayesianTools)
for i in range(N_particle):
    n = i % N_rows
    m = int(i/N_rows)
    cur_df = df.groupby("particle").get_group(i)        
    X_arr = cur_df[["x","y"]].values

    params = [T_stick,T_unstick,D,A]
#     S,XT,_=bayesianTools.viterbi_algorithm(X_arr,
#                                            T_stick, 
#                                            T_unstick, 
#                                            D, A, dt)
    S,XT,_ = bayesianTools.viterbi_algorithm(X_arr,T_stick,T_unstick,D,A,dt,True)
    fullState= [[S[j],X_arr[j],XT[j]] for j in range(N_steps)]

    S_true = cur_df.state.values
    XT_true = cur_df[["x_tether","y_tether"]].values

    fullState_true= [[S_true[j],X_arr[j],XT_true[j]] for j in range(N_steps)]

    cur_df_copy = cur_df.copy()
    cur_df_copy.loc[:,"state"]=S

    ax[n,m].plot(np.arange(N_steps),cur_df.state,'-')
    ax[n,m].plot(np.arange(N_steps),S,'--')
        

In [ ]:
from model import pack_model_params,model_trajectory_log_probability
model_params = pack_model_params(T_stick,T_unstick,D,A,dt)
S_tru = cur_df.state.values
reload(bayesianTools)
L_est=model_trajectory_log_probability(S,X_arr,model_params)
L_tru=model_trajectory_log_probability(S_tru,X_arr,model_params)
print(f"L_true: {L_tru}, L_est: {L_est}")


# Display Some Graphs

## Show trajectories

In [ ]:
startAtOrigin=False
if startAtOrigin:
    plotly_df=trajAnalysis.shift_traj_to_origin(df)
else:
    plotly_df = df.copy()
plotly_df.y *=-1
plotly_df["particle_state"] = plotly_df.particle.astype(str)+"_"+plotly_df.state.astype(str)
fig = px.line(plotly_df, x="x", y="y", color='particle_state',hover_data=['frame'],color_discrete_sequence=px.colors.qualitative.D3,)
fig.update_yaxes(scaleanchor = "x",scaleratio = 1,)
fig.show()

## Trajectories Animation:

In [ ]:
reload(displayData)
max_frame = 51
startAtOrigin=True
doneTrajNoMarker=True
dispLegend=False
if "Traj_fig" in locals() : plt.close(Traj_fig)
Traj_fig, Traj_ax = plt.subplots(figsize=(5,5))
drawnLines=displayData.plot_trajectories(df,curParticlePivotSize=5., t_end=max_frame, doneTrajNoMarker=doneTrajNoMarker,startAtOrigin=startAtOrigin,dispLegend=False, ax=Traj_ax, useDrawnLines=False, drawnLines=[],hideLines=True,axForVelocityPlot=[],particlesForFixedColoring = [])
def animateTraj(t_end):
    displayData.notebook_animate_traj(df,Traj_ax,t_end,max_frame,startAtOrigin=startAtOrigin,doneTrajNoMarker=doneTrajNoMarker,dispLegend=dispLegend,useDrawnLines=True,drawnLines=drawnLines,fps=0.,axForVelocityPlot=[],particlesForFixedColoring=[],curParticlePivotSize=5.)
ani = animation.FuncAnimation(Traj_fig, animateTraj, frames=max_frame); clear_output(wait=True); HTML(ani.to_jshtml())

## MSD Plot:

In [ ]:
default_groupby = "experiment"
if "MSD_fig" in locals() : plt.close(MSD_fig)
MSD_fig, MSD_ax = plt.subplots(figsize=(6, 4))
@interact(Log_Scale=True,group_by=["experiment","particle"],max_lagtime=(100,2000,10),N_sample_points = (2,101,1),equal_weight_per_particle=True)
#def update(group_by="experiment",Log_Scale=True,equal_weight_per_particle=True,step_lagtime=5,max_lagtime=100,):
def update(group_by=default_groupby,Log_Scale=False,equal_weight_per_particle=True,N_sample_points=21,max_lagtime=1000,):
    MSD_ax.cla()
    displayData.plot_MSD(df,isdfPadded=True, lagtime=np.unique(np.round(np.logspace(0,np.log10(max_lagtime),N_sample_points))), group_by=group_by, logscale=Log_Scale, ax=MSD_ax,eqParticleWeight=equal_weight_per_particle)
    MSD_ax.grid(True)

## Displacement Distribution $G(\Delta x,\Delta t)$:

In [ ]:
# set up plot
if "G_fig" in locals() : plt.close(G_fig)
G_fig, G_ax = plt.subplots(figsize=(6, 4))

dt_default = 3
xlim_default = 5.

@interact(dt=(1, min(200,df.frame.max()-1), 1),semiLogScale=True,direction=["xy", "x", "y"],group_by=["experiment","particle"],x_lim=(1., 30., 0.5),equal_weight_per_particle=True, clearAxes = True)
def update(dt=dt_default,x_lim=xlim_default,group_by="experiment",direction="xy",semiLogScale=True,equal_weight_per_particle=True,clearAxes = True):
    if clearAxes: G_ax.cla()        
    displayData.plot_G_dx_dt(df
                             ,isdfPadded=True, direction=direction, group_by=group_by, dt=dt, return_stats=False,semilogscale=semiLogScale,ax=G_ax,equal_particle_weight=equal_weight_per_particle)
    G_ax.set_xlim([-x_lim,x_lim])
    G_ax.set_ylim([1e-6,1e1]) if semiLogScale else G_ax.set_ylim([0,2.])   
    G_ax.grid(True)
    
#     sns.move_legend(G_ax,"lower left")
#     plt.setp(G_ax.get_legend().get_texts(), fontsize='7')  # for legend text
#     plt.setp(G_ax.get_legend().get_title(), fontsize='8')  # for legend title
